# Generate Summary Statistics for Studies
This notebook generates summary statistics (number of patients, number of visits, months followup, slope) from study data dictionaries

In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import joblib
import sys
from pathlib import Path

from analysis_utils import * 

In [3]:
# def initial_slope(df):
#     # Calculate slope as: (48-score at baseline)/time between onset and baseline in months.) 
#     xa_months = df['XA'][:,1]*12
#     slopes = -((48-df['YA'][:,1])/xa_months)
    
#     return(slopes)

In [4]:
def iqr(clist):
    """Calculate interquartile range"""
    q3, q1 = np.percentile(clist, [75 ,25])
    iqr = q3-q1
    return iqr
    
def generate_summary_statistics(data_path, data_name, proj_lis):
    """Calculate summ stats including average number of visits, months of followup, slope"""
    df_proj_sum = pd.DataFrame()
    study_type_dict = {'aals': 'Observational', 'ceft': 'Clinical Trial', 'emory': 'Observational', 'proact':'Clinical Trial', 'nathist':'Observational'}
    
    for proj in proj_lis:
        full_data_path = data_path / ('data_{}_{}.pkl').format(proj, data_name)
        assert full_data_path.exists(), 'File does not exist: {}'.format(full_data_path.resolve())
        cur_dat = joblib.load(data_path / ('data_{}_{}.pkl').format(proj, data_name))
        study_type = study_type_dict[proj]

        num_pats = len(cur_dat['SI'])
        XA = cur_dat['XA'][:,1:]
        YA = cur_dat['YA'][:,1:]
        num_visits = np.sum(~np.isnan(XA), axis=1) #exclude anchor onset
        med_num_visits = "{:.0f} ({:.0f})".format(np.median(num_visits), iqr(num_visits))

        months_follow = ((np.nanmax(XA, axis=1)-np.nanmin(XA, axis=1))*12)
        med_months_follow = "{:.2f} ({:.2f})".format(np.median(months_follow), iqr(months_follow))

        slope_df = calc_slope_mogp_data(cur_dat, include_anchor=True).dropna(how='any')['slope'] 
#         slope_df = initial_slope(cur_dat)
        med_slope = "{:.2f} ({:.2f})".format(np.median(slope_df), iqr(slope_df))

        df_proj_sum = df_proj_sum.append({'dataset': proj, 'study_type':study_type, 'total_pats': num_pats, 'med_num_vis': med_num_visits, 'med_month_follow':med_months_follow, 'med_slope':med_slope}, ignore_index=True)


    return df_proj_sum

def format_table(df_table, dataset_labels, column_labels):
    df_table = df_table.sort_values(by=['total_pats'], ascending=False)
    df_table['dataset'] = df_table['dataset'].map(dataset_labels)
    df_table = df_table.rename(columns=column_labels)
    return df_table


In [5]:
# Set formatting labels
projects = ['aals', 'ceft', 'emory', 'proact', 'nathist']
dat_lab = {'aals': 'AALS', 'ceft':'CEFT', 'emory':'EMORY', 'proact':'PRO-ACT', 'nathist':'NATHIST'}

col_lab = {'dataset':'Dataset', 'study_type':'Study Type', 'total_pats': 'Total No. Participants Included', 
                 'med_num_vis': 'Median (IQR) No. Visits', 'med_month_follow': 'Median (IQR) Months Followed', 
                 'med_slope': 'Median (IQR) ALSFRS-R Slope', 'male':'No. (%) Male', 'female':'No. (%) Female',
            'limb_onset':'No. (%) Limb Onset',  'bulbar_onset':'No. (%) Bulbar Onset', 'age_onset': 'Median (IQR) Age of Onset', 
           'diag_delay':'Median (IQR) Diagnostic Delay'
          }

In [6]:
# Generate summary for ALSFRS-R models with 3 or more visits
data_min3 = Path('data/model_data/1_alsfrsr_all')
sum_data_min3 = generate_summary_statistics(data_min3, 'min3_alsfrst', projects)
sum_data_min3 = format_table(sum_data_min3, dat_lab, col_lab)

sum_data_min3

# Save table
sum_data_min3.to_csv('reports/table_study_summary_stats_median.csv', index=False)

,Dataset,Study Type,Total No. Participants Included,Median (IQR) No. Visits,Median (IQR) Months Followed,Median (IQR) ALSFRS-R Slope
3,PRO-ACT,Clinical Trial,2923.0,9 (7),11.95 (4.63),-0.67 (0.64)
4,NATHIST,Observational,907.0,5 (3),17.24 (19.56),-0.65 (0.62)
1,CEFT,Clinical Trial,476.0,9 (7),16.80 (14.40),-0.84 (0.63)
0,AALS,Observational,456.0,5 (3),13.79 (10.55),-0.55 (0.62)
2,EMORY,Observational,399.0,4 (3),15.04 (13.76),-0.89 (0.67)


In [7]:
# Generate summary for PROACT/CEFT models with 10 or more visits
data_min4 = Path('data/model_data/2_sparsity_prediction/prediction')
sum_data_min4 = generate_summary_statistics(data_min4, 'min4_predict_full', ['proact', 'ceft', 'aals', 'emory', 'nathist'])
sum_data_min4 = format_table(sum_data_min4, dat_lab, col_lab)
sum_data_min4.insert(1, 'Inclusion Criteria', '>= 4 visits')

data_min10 = Path('data/model_data/2_sparsity_prediction/sparsity')
sum_data_min10 = generate_summary_statistics(data_min10, 'min10_sparse_full', ['proact', 'ceft', 'nathist'])
sum_data_min10 = format_table(sum_data_min10, dat_lab, col_lab)
sum_data_min10.insert(1, 'Inclusion Criteria', '>= 10 visits')

sum_data_agg = sum_data_min4.append(sum_data_min10, ignore_index=True)

sum_data_agg

# Save table
sum_data_agg.to_csv('reports/supp_table_study_summ_stats_predsparse_median.csv', index=False)

,Dataset,Inclusion Criteria,Study Type,Total No. Participants Included,Median (IQR) No. Visits,Median (IQR) Months Followed,Median (IQR) ALSFRS-R Slope
0,PRO-ACT,>= 4 visits,Clinical Trial,2814.0,9 (8),11.95 (4.43),-0.66 (0.63)
1,NATHIST,>= 4 visits,Observational,714.0,6 (3),20.00 (20.66),-0.61 (0.62)
2,CEFT,>= 4 visits,Clinical Trial,453.0,10 (7),18.00 (13.20),-0.81 (0.59)
3,AALS,>= 4 visits,Observational,341.0,5 (2),15.86 (11.49),-0.49 (0.57)
4,EMORY,>= 4 visits,Observational,283.0,5 (3),18.95 (14.94),-0.86 (0.62)
5,PRO-ACT,>= 10 visits,Clinical Trial,1327.0,14 (5),13.46 (3.97),-0.66 (0.58)
6,CEFT,>= 10 visits,Clinical Trial,228.0,13 (5),25.20 (10.80),-0.69 (0.46)
7,NATHIST,>= 10 visits,Observational,132.0,12 (6),44.76 (24.52),-0.42 (0.33)


# Generate extended version of Table 1

In [8]:
# AALS
aals_diag = pd.read_csv('data/raw_data/aals/v_NB_IATI_AALSHXFX.csv').set_index('SubjectUID').apply(pd.to_numeric, errors='coerce')
aals_dem = pd.read_csv('data/raw_data/aals/v_NB_IATI_Demographics.csv').set_index('SubjectUID').apply(pd.to_numeric, errors='coerce')
aals_diag['diag_delay']=(aals_diag['diagdt'].dropna()-aals_diag['onsetdt'].dropna())/365.4

aals_dem = aals_dem[['dob', 'sex', 'age']]
aals_diag = aals_diag[['diag_delay', 'hxblb', 'hxli', 'hxax', 'onsetdt']]

aals_stat = aals_dem.join(aals_diag)
aals_stat['age_onset'] = ((aals_stat['onsetdt']-aals_stat['dob'])/365.4).dropna() #Unit: Years
aals_stat['sex'] = aals_stat['sex'].map({1:'Male', 2:'Female'})
aals_stat['proj']='aals'
assert aals_stat.index.is_unique

# NATHIST
nh_diag = pd.read_csv('data/raw_data/nathist/v_NB_CLIN_NALSHXFX.csv').set_index('Neurostamp').apply(pd.to_numeric, errors='coerce')
nh_dem = pd.read_csv('data/raw_data/nathist/v_NB_CLIN_Demographics.csv').set_index('Neurostamp').apply(pd.to_numeric, errors='coerce')
nh_diag['diag_delay']=(nh_diag['diagdt'].dropna()-nh_diag['onsetdt'].dropna())/365.4

nh_dem = nh_dem[['dob', 'sex', 'age']]
nh_diag = nh_diag[['diag_delay', 'hxblb', 'hxli', 'hxax', 'onsetdt']]
nh_diag = nh_diag.reset_index().groupby('Neurostamp').mean()
nh_stat = nh_dem.join(nh_diag)
nh_stat['age_onset'] = ((nh_stat['onsetdt']-nh_stat['dob'])/365.4).dropna() #Unit: Years
nh_stat['sex'] = nh_stat['sex'].map({1:'Male', 2:'Female'})
nh_stat['proj']='nathist'
assert nh_stat.index.is_unique

# PROACT
df_alshist = pd.read_csv('data/raw_data/proact/AlsHistory.csv').set_index('subject_id')
df_demo = pd.read_csv('data/raw_data/proact/demographics.csv').set_index('subject_id')

cond1 = df_alshist['Site_of_Onset___Bulbar'] == 1
cond2 = df_alshist['Site_of_Onset'] == 'Onset: Bulbar'
df_alshist['Bulbar_Onset_Merge'] = np.where(cond1 | cond2, 1, np.nan)

cond1 = df_alshist['Site_of_Onset___Limb'] == 1
cond2 = df_alshist['Site_of_Onset'] == 'Onset: Limb'
df_alshist['Limb_Onset_Merge'] = np.where(cond1 | cond2, 1, np.nan)

df_alshist_onset = df_alshist[['Bulbar_Onset_Merge', 'Limb_Onset_Merge']]
df_alshist_onset = df_alshist_onset.reset_index().groupby('subject_id').max()

df_alshist_sub = df_alshist[['Onset_Delta']].reset_index().drop_duplicates().dropna().set_index('subject_id')
df_alshist = df_alshist_onset.join(df_alshist_sub, how='outer')
proact_stat = df_alshist.join(df_demo)
proact_stat = proact_stat[['Sex', 'Age', 'Date_of_Birth', 'Bulbar_Onset_Merge', 'Limb_Onset_Merge', 'Onset_Delta']]
proact_stat['age_onset']=proact_stat['Age']+(proact_stat['Onset_Delta']/365.4)
proact_stat.rename(columns={'Sex':'sex', 'Bulbar_Onset_Merge':'hxblb', 'Limb_Onset_Merge':'hxli'}, inplace=True)
proact_stat['proj']='proact'
assert proact_stat.index.is_unique

# CEFT
ceft_dem = pd.read_csv('data/processed_data/ceft/demo.csv').set_index('study_id')
ceft_mhas = pd.read_csv('data/processed_data/ceft/mhas.csv').set_index('study_id')
ceft_dem = ceft_dem[['sex']]
ceft_mhas = ceft_mhas[['siteofonset', 'age_at_ALS_DATE_SYM', 'age_at_BASELINE', 'age_at_DATE_OF_DIAG']]
ceft_mhas['diag_delay']=ceft_mhas['age_at_DATE_OF_DIAG']-ceft_mhas['age_at_ALS_DATE_SYM']
ceft_stat = ceft_dem.join(ceft_mhas)
ceft_stat.rename(columns={'age_at_ALS_DATE_SYM':'age_onset'}, inplace=True)
ceft_stat['sex'] = ceft_stat['sex'].map({0:'Male', 1:'Female'})
ceft_stat['siteofonset'] = ceft_stat['siteofonset'].map({0:'limb', 1:'bulbar', 2:'both'})
ceft_stat['hxli'] = np.where((ceft_stat['siteofonset']=='limb') | (ceft_stat['siteofonset']=='both'), 1, 0)
ceft_stat['hxblb'] = np.where((ceft_stat['siteofonset']=='bulbar') | (ceft_stat['siteofonset']=='both'), 1, 0)
ceft_stat['proj']='ceft'
assert ceft_stat.index.is_unique


## EMORY
emory_stat = pd.read_excel('data/raw_data/emory/emory_deidentified_04012020.xlsx')
emory_stat = emory_stat[['Patient ID', 'Gender', 'Age Onset']].drop_duplicates().set_index('Patient ID')
emory_stat.rename(columns={'Gender':'sex', 'Age Onset':'age_onset'}, inplace=True)
emory_stat['proj']='emory'
assert emory_stat.index.is_unique


In [9]:
df_extend = pd.DataFrame()
df_extend = df_extend.append([aals_stat, nh_stat, proact_stat, ceft_stat, emory_stat])
df_extend = df_extend[['proj', 'sex', 'age_onset', 'diag_delay', 'hxblb', 'hxli']]
df_extend['si_ind']=(df_extend.index).astype(str)+'_'+df_extend['proj']

df_extend['sex'].replace({'male':'Male'}, inplace=True)

In [10]:
df_extend.to_csv('data/processed_data/pat_clin_summ_stats.csv')

In [11]:
def extend_statistics(data_path, data_name, proj_lis, ext):
    """Calculate summ stats including average number of visits, months of followup, slope"""
    df_proj_sum = pd.DataFrame()    
    for proj in proj_lis:
        full_data_path = data_path / ('data_{}_{}.pkl').format(proj, data_name)
        assert full_data_path.exists(), 'File does not exist: {}'.format(full_data_path.resolve())
        cur_dat = joblib.load(data_path / ('data_{}_{}.pkl').format(proj, data_name))

        SI = cur_dat['SI']
        cext = ext[ext['si_ind'].isin(SI)]
        m_stat = '{} ({:.1f})'.format(cext['sex'].value_counts()['Male'], (cext['sex'].value_counts()['Male']/len(SI))*100)
        f_stat = '{} ({:.1f})'.format(cext['sex'].value_counts()['Female'], (cext['sex'].value_counts()['Female']/len(SI))*100)
        limb_stat = '{:.0f} ({:.1f})'.format(cext['hxli'].sum(), (cext['hxli'].sum()/len(SI))*100)
        bulbar_stat = '{:.0f} ({:.1f})'.format(cext['hxblb'].sum(), (cext['hxblb'].sum()/len(SI))*100)
        if cext['hxli'].sum() == 0:
            limb_stat = None
        if cext['hxblb'].sum() == 0:
            bulbar_stat = None
        try:
            age_onset_stat = "{:.2f} ({:.2f})".format(np.median(cext['age_onset'].dropna()), iqr(cext['age_onset'].dropna()))
        except IndexError:
            age_onset_stat = None
        try:
            diag_delay_stat = "{:.2f} ({:.2f})".format(np.median(cext['diag_delay'].dropna()), iqr(cext['diag_delay'].dropna()))
        except IndexError:
            diag_delay_stat = None
        
        
        df_proj_sum = df_proj_sum.append({'dataset': proj, 'male':m_stat, 'female':f_stat, 'limb_onset':limb_stat, 
                                         'bulbar_onset':bulbar_stat, 'age_onset':age_onset_stat, 
                                         'diag_delay':diag_delay_stat}, ignore_index=True)


    return df_proj_sum

In [12]:
data_min3 = Path('data/model_data/1_alsfrsr_all')
df_ext = extend_statistics(data_min3, 'min3_alsfrst', projects, df_extend)
sum_data_min3 = generate_summary_statistics(data_min3, 'min3_alsfrst', projects)

 /Users/divyar/Documents/Grad - Divya/Research/mogp/mogpvenv-3.7.3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:Mean of empty slice.
 /Users/divyar/Documents/Grad - Divya/Research/mogp/mogpvenv-3.7.3/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:invalid value encountered in double_scalars


In [13]:
df_ext_merge = df_ext.merge(sum_data_min3,  on='dataset')
df_ext_merge_format = format_table(df_ext_merge, dat_lab, col_lab)
df_ext_merge_format.to_csv('reports/supp_table_extend_summary_stats_median.csv', index=False)


In [14]:
df_ext_merge_format

,Dataset,No. (%) Male,No. (%) Female,No. (%) Limb Onset,No. (%) Bulbar Onset,Median (IQR) Age of Onset,Median (IQR) Diagnostic Delay,Study Type,Total No. Participants Included,Median (IQR) No. Visits,Median (IQR) Months Followed,Median (IQR) ALSFRS-R Slope
3,PRO-ACT,1838 (62.9),1085 (37.1),2001 (68.5),589 (20.2),55.10 (16.11),None,Clinical Trial,2923.0,9 (7),11.95 (4.63),-0.67 (0.64)
4,NATHIST,537 (59.2),368 (40.6),621 (68.5),239 (26.4),61.73 (14.24),0.99 (1.06),Observational,907.0,5 (3),17.24 (19.56),-0.65 (0.62)
1,CEFT,289 (60.7),187 (39.3),377 (79.2),108 (22.7),54.70 (15.12),0.80 (0.70),Clinical Trial,476.0,9 (7),16.80 (14.40),-0.84 (0.63)
0,AALS,287 (62.9),169 (37.1),344 (75.4),111 (24.3),57.85 (13.96),0.98 (1.02),Observational,456.0,5 (3),13.79 (10.55),-0.55 (0.62)
2,EMORY,233 (58.4),166 (41.6),None,None,61.09 (16.40),None,Observational,399.0,4 (3),15.04 (13.76),-0.89 (0.67)
